#Chart2Go 


In [27]:
#!/usr/bin/env python3
"""
NQ Futures Dashboard Generator - 15-MINUTE CANDLESTICK VERSION
Generates an HTML page with 15-minute candlestick chart and dashboard data for NQ futures
"""

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import os

def get_nq_data():
    """Fetch NQ futures data from Yahoo Finance"""
    ticker = "NQ=F"
    
    try:
        ticker_obj = yf.Ticker(ticker)
        # Get 15-minute data for the last week
        minute_data = ticker_obj.history(period="7d", interval="15m")
        # Get daily data for moving averages (need more history)
        daily_data = ticker_obj.history(period="1y", interval="1d")
        return minute_data, daily_data
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None, None

def create_sample_data():
    """Create sample NQ futures data"""
    import numpy as np
    
    # Create 15-minute intervals for 1 week (7 days * 24 hours * 4 intervals = 672 periods)
    # But markets are only open ~6.5 hours, so about 26 intervals per day * 7 days = 182 periods
    dates = pd.date_range(start='2024-06-03', end='2024-06-10', freq='15T')
    # Filter to market hours (9:30 AM to 4:00 PM ET)
    market_dates = []
    for date in dates:
        if 9.5 <= date.hour + date.minute/60 <= 16:
            market_dates.append(date)
    
    np.random.seed(42)
    
    base_price = 18000
    minute_price_data = []
    current_price = base_price
    
    for i, date in enumerate(market_dates[:200]):  # Limit to ~200 candles for better display
        # Smaller movements for 15-minute intervals
        change = np.random.normal(0, 20)
        current_price += change
        
        open_price = current_price
        high_price = open_price + abs(np.random.normal(0, 15))
        low_price = open_price - abs(np.random.normal(0, 15))
        close_price = low_price + (high_price - low_price) * np.random.random()
        
        minute_price_data.append({
            'Open': open_price,
            'High': high_price,
            'Low': low_price,
            'Close': close_price,
            'Volume': int(np.random.normal(1000, 200))
        })
        
        current_price = close_price
    
    minute_data = pd.DataFrame(minute_price_data, index=market_dates[:200])
    
    # Create daily data for moving averages
    daily_dates = pd.date_range(start='2023-06-01', end='2024-06-10', freq='D')
    daily_price_data = []
    current_price = base_price - 1000  # Start lower for trend
    
    for date in daily_dates:
        daily_change = np.random.normal(5, 50)
        current_price += daily_change
        
        open_price = current_price
        high_price = open_price + abs(np.random.normal(0, 40))
        low_price = open_price - abs(np.random.normal(0, 40))
        close_price = low_price + (high_price - low_price) * np.random.random()
        
        daily_price_data.append({
            'Open': open_price,
            'High': high_price,
            'Low': low_price,
            'Close': close_price,
            'Volume': int(np.random.normal(50000, 10000))
        })
        
        current_price = close_price
    
    daily_data = pd.DataFrame(daily_price_data, index=daily_dates)
    
    return minute_data, daily_data

def calculate_moving_averages(daily_data, minute_data):
    """Calculate 20, 50, and 200 day moving averages"""
    if daily_data is None or len(daily_data) < 200:
        return None, None, None
    
    # Calculate moving averages on daily data
    ma20 = daily_data['Close'].rolling(window=20).mean()
    ma50 = daily_data['Close'].rolling(window=50).mean()
    ma200 = daily_data['Close'].rolling(window=200).mean()
    
    # Get the latest values to display as horizontal lines on the 15-min chart
    latest_ma20 = ma20.iloc[-1] if not ma20.isna().iloc[-1] else None
    latest_ma50 = ma50.iloc[-1] if not ma50.isna().iloc[-1] else None
    latest_ma200 = ma200.iloc[-1] if not ma200.isna().iloc[-1] else None
    
    return latest_ma20, latest_ma50, latest_ma200

def calculate_range(data, periods):
    """Calculate average range for given periods"""
    if data is None or len(data) < periods:
        return 0.0
    
    recent_data = data.tail(periods)
    ranges = recent_data['High'] - recent_data['Low']
    return float(round(ranges.mean(), 2))

def create_candlestick_html(minute_data, ma20, ma50, ma200):
    """Create CSS candlestick chart HTML without moving average lines"""
    # Get data for display
    chart_data = minute_data.tail(100)  # Show last 100 15-minute candles
    
    # Calculate price range for scaling
    min_price = chart_data[['Open', 'High', 'Low', 'Close']].min().min()
    max_price = chart_data[['Open', 'High', 'Low', 'Close']].max().max()
    price_range = max_price - min_price
    
    candlesticks_html = ""
    
    # Create candlesticks with time labels every 10th candle
    for i, (timestamp, row) in enumerate(chart_data.iterrows()):
        open_price = row['Open']
        high_price = row['High']
        low_price = row['Low']
        close_price = row['Close']
        
        # Determine if bullish (green) or bearish (red)
        is_bullish = close_price >= open_price
        color = "#00ff88" if is_bullish else "#ff4040"
        
        # Calculate positions (inverted because CSS top:0 is at top)
        high_pos = ((max_price - high_price) / price_range) * 100
        low_pos = ((max_price - low_price) / price_range) * 100
        
        if is_bullish:
            body_top = ((max_price - close_price) / price_range) * 100
            body_bottom = ((max_price - open_price) / price_range) * 100
        else:
            body_top = ((max_price - open_price) / price_range) * 100
            body_bottom = ((max_price - close_price) / price_range) * 100
        
        body_height = body_bottom - body_top
        if body_height < 0.5:  # Ensure minimum visibility for doji candles
            body_height = 0.5
        
        # Show time label every 10th candle or if it's a new day
        show_time = (i % 10 == 0) or (i > 0 and chart_data.index[i].date() != chart_data.index[i-1].date())
        time_label = ""
        if show_time:
            if chart_data.index[i].time() == pd.Timestamp('09:30:00').time():
                # Show date for market open
                time_label = f'<div class="time-label major">{timestamp.strftime("%m/%d")}</div>'
            else:
                # Show time for other intervals
                time_label = f'<div class="time-label">{timestamp.strftime("%H:%M")}</div>'
        
        # Create candlestick HTML with thicker candles
        candlesticks_html += f'''
            <div class="candlestick" style="left: {i * 0.95 + 0.5}%;">
                <div class="wick" style="top: {high_pos}%; height: {low_pos - high_pos}%;"></div>
                <div class="body" style="top: {body_top}%; height: {body_height}%; background-color: {color};"></div>
                {time_label}
                <div class="price-tooltip">
                    T: {timestamp.strftime('%m/%d %H:%M')}<br>
                    O: ${open_price:.0f}<br>
                    H: ${high_price:.0f}<br>
                    L: ${low_price:.0f}<br>
                    C: ${close_price:.0f}
                </div>
            </div>
        '''
    
    return candlesticks_html

def generate_html():
    """Generate the HTML dashboard"""
    
    # Get data
    minute_data, daily_data = get_nq_data()
    
    if minute_data is None:
        print("Failed to fetch data. Using sample data.")
        minute_data, daily_data = create_sample_data()
    
    # Calculate metrics
    current_price = float(round(minute_data['Close'].iloc[-1], 2))
    current_volume = int(minute_data['Volume'].iloc[-1])
    day_range = calculate_range(minute_data, 26)  # ~1 day of 15-min candles
    week_range = calculate_range(minute_data, 182)  # ~1 week
    hour_range = calculate_range(minute_data, 4)  # 4 15-min periods = 1 hour
    min15_range = calculate_range(minute_data, 1)
    
    # Calculate moving averages
    ma20, ma50, ma200 = calculate_moving_averages(daily_data, minute_data)
    
    # Format moving averages for display
    ma20_display = f"${ma20:.0f}" if ma20 else "N/A"
    ma50_display = f"${ma50:.0f}" if ma50 else "N/A"
    ma200_display = f"${ma200:.0f}" if ma200 else "N/A"
    
    # Create candlestick chart
    candlesticks_html = create_candlestick_html(minute_data, ma20, ma50, ma200)
    
    last_updated = datetime.now().strftime('%Y-%m-%d %H:%M:%S EST')
    
    html = f'''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>NQ Futures Dashboard</title>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        body {{
            background: linear-gradient(135deg, #0c0c0c 0%, #1a1a1a 100%);
            color: #e0e0e0;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            min-height: 100vh;
            padding: 20px;
        }}
        
        .container {{
            max-width: 1400px;
            margin: 0 auto;
        }}
        
        .header {{
            text-align: center;
            margin-bottom: 30px;
            padding: 20px 0;
            border-bottom: 2px solid #333;
        }}
        
        .header h1 {{
            font-size: 2.5em;
            color: #00ff88;
            text-shadow: 0 0 10px rgba(0, 255, 136, 0.3);
            margin-bottom: 10px;
        }}
        
        .subtitle {{
            font-size: 1.1em;
            color: #888;
        }}
        
        .dashboard {{
            display: grid;
            grid-template-columns: 1fr 350px;
            gap: 20px;
            margin-bottom: 20px;
        }}
        
        .chart-container {{
            background: rgba(20, 20, 20, 0.8);
            border-radius: 12px;
            padding: 20px;
            border: 1px solid #333;
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
        }}
        
        .chart-title {{
            color: #00ff88;
            font-size: 1.2em;
            margin-bottom: 15px;
            text-align: center;
        }}
        
        .chart-wrapper {{
            position: relative;
            height: 500px;
            background: rgba(10, 10, 10, 0.5);
            border: 1px solid #333;
            border-radius: 8px;
            overflow: hidden;
            margin-bottom: 50px;
        }}
        
        .candlestick {{
            position: absolute;
            width: 0.8%;
            height: 100%;
        }}
        
        .wick {{
            position: absolute;
            left: 50%;
            transform: translateX(-50%);
            width: 2px;
            background-color: #888;
        }}
        
        .body {{
            position: absolute;
            left: 10%;
            width: 80%;
            border: 1px solid rgba(255, 255, 255, 0.3);
            min-height: 2px;
        }}
        
        .candlestick:hover .price-tooltip {{
            display: block;
        }}
        
        .price-tooltip {{
            display: none;
            position: absolute;
            top: -70px;
            left: 50%;
            transform: translateX(-50%);
            background: rgba(0, 0, 0, 0.9);
            color: #fff;
            padding: 8px;
            border-radius: 4px;
            font-size: 0.8em;
            white-space: nowrap;
            z-index: 100;
            border: 1px solid #333;
        }}
        
        .time-label {{
            position: absolute;
            bottom: -35px;
            left: 50%;
            transform: translateX(-50%);
            font-size: 0.7em;
            color: #888;
            white-space: nowrap;
            font-weight: normal;
        }}
        
        .time-label.major {{
            font-size: 0.8em;
            color: #aaa;
            font-weight: bold;
            bottom: -45px;
        }}
        
        .ma-legend {{
            margin-top: 15px;
            text-align: center;
            font-size: 0.85em;
            padding: 10px;
            background: rgba(30, 30, 30, 0.5);
            border-radius: 6px;
            border: 1px solid #444;
        }}
        
        .ma-legend-item {{
            display: inline-block;
            margin: 0 15px;
            color: #ccc;
        }}
        
        .ma-color {{
            display: inline-block;
            width: 20px;
            height: 3px;
            margin-right: 5px;
            vertical-align: middle;
        }}
        
        .ma20-color {{
            background-color: #ffaa00;
        }}
        
        .ma50-color {{
            background-color: #4da6ff;
        }}
        
        .ma200-color {{
            background-color: #ff6b6b;
        }}
        
        .stats-panel {{
            background: rgba(20, 20, 20, 0.8);
            border-radius: 12px;
            padding: 20px;
            border: 1px solid #333;
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
        }}
        
        .stat-card {{
            background: rgba(30, 30, 30, 0.6);
            border-radius: 8px;
            padding: 15px;
            margin-bottom: 15px;
            border-left: 4px solid #00ff88;
            transition: all 0.3s ease;
        }}
        
        .stat-card:hover {{
            background: rgba(40, 40, 40, 0.8);
            transform: translateX(5px);
        }}
        
        .stat-label {{
            font-size: 0.9em;
            color: #aaa;
            margin-bottom: 5px;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }}
        
        .stat-value {{
            font-size: 1.4em;
            font-weight: bold;
            color: #fff;
        }}
        
        .price {{
            color: #00ff88;
            font-size: 1.6em !important;
        }}
        
        .volume {{
            color: #4da6ff;
        }}
        
        .range {{
            color: #ffaa00;
        }}
        
        .footer {{
            text-align: center;
            margin-top: 30px;
            padding-top: 20px;
            border-top: 1px solid #333;
            color: #666;
            font-size: 0.9em;
        }}
        
        .legend {{
            margin-top: 10px;
            text-align: center;
            font-size: 0.9em;
        }}
        
        .legend-item {{
            display: inline-block;
            margin: 0 15px;
            color: #888;
        }}
        
        .legend-color {{
            display: inline-block;
            width: 12px;
            height: 12px;
            margin-right: 5px;
            vertical-align: middle;
        }}
        
        .bullish {{
            background-color: #00ff88;
        }}
        
        .bearish {{
            background-color: #ff4040;
        }}
        
        @media (max-width: 1200px) {{
            .dashboard {{
                grid-template-columns: 1fr;
            }}
            
            .stats-panel {{
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                gap: 15px;
            }}
            
            .stat-card {{
                margin-bottom: 0;
            }}
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>NQ FUTURES DASHBOARD</h1>
            <div class="subtitle">NASDAQ 100 E-mini Futures • 15-Minute Candlesticks</div>
        </div>
        
        <div class="dashboard">
            <div class="chart-container">
                <div class="chart-title">NQ Futures - 15-Minute Candles (Last Week)</div>
                <div class="chart-wrapper">
                    {candlesticks_html}
                </div>
                <div class="legend">
                    <div class="legend-item">
                        <span class="legend-color bullish"></span>Bullish (Close ≥ Open)
                    </div>
                    <div class="legend-item">
                        <span class="legend-color bearish"></span>Bearish (Close < Open)
                    </div>
                </div>
                <div class="ma-legend">
                    <div class="ma-legend-item">
                        <span class="ma-color ma20-color"></span>MA20: {ma20_display}
                    </div>
                    <div class="ma-legend-item">
                        <span class="ma-color ma50-color"></span>MA50: {ma50_display}
                    </div>
                    <div class="ma-legend-item">
                        <span class="ma-color ma200-color"></span>MA200: {ma200_display}
                    </div>
                </div>
            </div>
            
            <div class="stats-panel">
                <div class="stat-card">
                    <div class="stat-label">Current Price</div>
                    <div class="stat-value price">${current_price:,.2f}</div>
                </div>
                
                <div class="stat-card">
                    <div class="stat-label">Volume</div>
                    <div class="stat-value volume">{current_volume:,}</div>
                </div>
                
                <div class="stat-card">
                    <div class="stat-label">Day Range (Avg)</div>
                    <div class="stat-value range">{day_range} pts</div>
                </div>
                
                <div class="stat-card">
                    <div class="stat-label">Week Range (Avg)</div>
                    <div class="stat-value range">{week_range} pts</div>
                </div>
                
                <div class="stat-card">
                    <div class="stat-label">Hour Range (Avg)</div>
                    <div class="stat-value range">{hour_range} pts</div>
                </div>
                
                <div class="stat-card">
                    <div class="stat-label">15min Range (Avg)</div>
                    <div class="stat-value range">{min15_range} pts</div>
                </div>
            </div>
        </div>
        
        <div class="footer">
            <p>Last Updated: {last_updated} | Data provided by Yahoo Finance</p>
            <p>⚠️ This is for educational purposes only. Not financial advice.</p>
            <p>💡 Hover over candlesticks to see OHLC data</p>
        </div>
    </div>
</body>
</html>'''
    
    return html

def main():
    """Main function"""
    print("Generating NQ Futures Dashboard...")
    
    html_content = generate_html()
    
    filename = f"nq_dashboard_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    full_path = os.path.abspath(filename)
    file_url = f"file://{full_path}"
    
    print(f"\n✅ Dashboard generated successfully!")
    print(f"📁 File location: {full_path}")
    print(f"🔗 Click to open: {file_url}")
    print(f"\nOr copy and paste this link into your browser:")
    print(f"{file_url}")
    print("\n" + "="*60)

if __name__ == "__main__":
    try:
        import yfinance
        import pandas
        import numpy
    except ImportError:
        print("Installing required packages...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "yfinance", "pandas", "numpy"])
    
    main()

Generating NQ Futures Dashboard...
Error fetching data: Too Many Requests. Rate limited. Try after a while.
Failed to fetch data. Using sample data.

✅ Dashboard generated successfully!
📁 File location: c:\Users\Wolfrank\Desktop\DDesktop\CodeWolf\QuantTrading-1\Tools\Chart2go\nq_dashboard_20250610_155326.html
🔗 Click to open: file://c:\Users\Wolfrank\Desktop\DDesktop\CodeWolf\QuantTrading-1\Tools\Chart2go\nq_dashboard_20250610_155326.html

Or copy and paste this link into your browser:
file://c:\Users\Wolfrank\Desktop\DDesktop\CodeWolf\QuantTrading-1\Tools\Chart2go\nq_dashboard_20250610_155326.html



C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_16504\595845818.py:33: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates = pd.date_range(start='2024-06-03', end='2024-06-10', freq='15T')
